## Проверка исходных данных
Исходные данные, хранятся в локальном файле parquet. Набор данных фиксирует почасовую статистику водителя в приложении для совместного использования поездок.

In [1]:
import pandas as pd
pd.read_parquet("feature_repo/data/driver_stats.parquet")

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2025-06-07 07:00:00+00:00,1005,0.231275,0.072127,555,2025-06-22 07:14:56.270
1,2025-06-07 08:00:00+00:00,1005,0.344026,0.625116,725,2025-06-22 07:14:56.270
2,2025-06-07 09:00:00+00:00,1005,0.163056,0.650430,656,2025-06-22 07:14:56.270
3,2025-06-07 10:00:00+00:00,1005,0.168519,0.835975,99,2025-06-22 07:14:56.270
4,2025-06-07 11:00:00+00:00,1005,0.375509,0.562910,231,2025-06-22 07:14:56.270
...,...,...,...,...,...,...
1802,2025-06-22 05:00:00+00:00,1001,0.219542,0.435670,8,2025-06-22 07:14:56.270
1803,2025-06-22 06:00:00+00:00,1001,0.173728,0.825187,40,2025-06-22 07:14:56.270
1804,2021-04-12 07:00:00+00:00,1001,0.012422,0.535634,37,2025-06-22 07:14:56.270
1805,2025-06-14 19:00:00+00:00,1003,0.117150,0.657742,623,2025-06-22 07:14:56.270


## Регистрация определения объектов и развание хранилищ
Команда **apply** сканирует файлы python в текущем каталоге на предмет определения представлений объектов/сущностей, регистрирует объекты и развертывает инфраструктуру. Тут она считывает **example_repo.py** и настраивает таблицы интернет-магазина SQLite. Обратите внимание, что мы указали SQLite в качестве интернет-магазина по умолчанию, настроив **online_store** в файле **feature_store.yaml**.

In [2]:
%cd feature_repo
!uv run feast apply

/data/projects/otus-mlops/04-feature-store/feature_repo
/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(
/data/projects/otus-mlops/04-feature-store/feature_repo/example_repo.py:27: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity 'driver'.
  driver = Entity(name="driver", join_keys=["driver_id"])
Applying changes for project repo
/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/feast/feature_store.py:580: Runtime

## Генерация обучающих данных
Для обучения модели нам нужны свойства и метки. Часто эти данные хранятся отдельно. Feast может помочь сгенерировать объекты, соответствующие меткам.
Feast ожидает список объектов (например, идентификаторы драйверов) и временные метки. Feast объединит соответствующие таблицы для создания векторов объектов. 
Временные метки включаются, чтобы в модели использовались характеристики для одного и того же драйвера в разное время.

In [3]:
from datetime import datetime
import pandas as pd

from feast import FeatureStore


entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

store = FeatureStore(repo_path=".")

training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

training_df.head()

/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2021-04-12 10:59:42+00:00,1,1,10,0.012422,0.535634,37,1.012422,10.012422
1,1002,2021-04-12 08:12:10+00:00,5,2,20,0.470895,0.148993,945,2.470895,20.470895
2,1003,2021-04-12 16:40:26+00:00,3,3,30,0.646589,0.687586,890,3.646589,30.646589


## Выполнение офлайн-вывода (пакетная оценка)
Для поддержки пакетной модели необходимо сгенерировать признаки с помощью **get_historical_features**, используя текущую временную метку.

In [4]:
entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

training_df.head()

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2025-06-22 03:07:03.570107+00:00,1,1,10,0.459428,0.796430,492,1.459428,10.459428
1,1002,2025-06-22 03:07:03.570107+00:00,5,2,20,0.419348,0.687892,194,2.419348,20.419348
2,1003,2025-06-22 03:07:03.570107+00:00,3,3,30,0.476563,0.170061,477,3.476563,30.476563


## Добавление пакетного признака 
Команда **materialize_incremental** сериализует все новые признаки с момента последнего вызова **materialize** или с момента указанной даты.

In [5]:
!uv run feast materialize-incremental $(date -u +"%Y-%m-%dT%H:%M:%S")

/usr/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=1044927) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "vector_enabled" in "SqliteOnlineStoreConfig" shadows an attribute in parent "VectorStoreConfig"
  warnings.warn(
/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(
Materializing 2 feature views to 2025-06-22 03:07:03+00:00 into the sqlite online store.

driver_hourly_stats from 2025-06-22 02:53:54+00:00 to 2025-06-22 03:07:03+00:00:
100%|████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 993.72it/s]
driver_hourly_stats_fresh from 2025-06-22 02:53:54+00:00 to 2025-06-22 03:07:03+00:00:
100%|███████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1832.69it/s]


## Извлечение векторов признаков для вывода
Во время вывода нам нужно быстро считать последние значения признаков для разных драйверов (которые могли бы существовать только в пакетных источниках) из онлайн-хранилища признаков с помощью **get_online_features**. Затем эти векторы признаков можно передать в модель.

In [6]:
from feast import FeatureStore

store = FeatureStore(repo_path=".")

feature_vector = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[
        # {join_key: entity_value}
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

feature_vector

/data/projects/otus-mlops/04-feature-store/.venv/lib/python3.12/site-packages/feast/repo_config.py:268: DeprecationWarning: The serialization version 2 and below will be deprecated in the next release. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


{'driver_id': [1004, 1005],
 'avg_daily_trips': [553, 762],
 'conv_rate': [0.7619121670722961, 0.45416760444641113],
 'acc_rate': [0.8696280121803284, 0.05302406847476959]}

## Использование сервиса для получения онлайн-признаков.
Также можно использовать службы для управления несколькими признаками  Хранилище также можно использовать для извлечения как онлайн, так и исторических функций с использованием того же API.

Служба **driver_activity_v1** извлекает все признаки из **driver_hourly_stats** представления:

In [7]:
from feast import FeatureStore
feature_store = FeatureStore('.')  # Initialize the feature store

feature_service = feature_store.get_feature_service("driver_activity_v1")
feature_service
feature_vector = feature_store.get_online_features(
    features=feature_service,
    entity_rows=[
        {"driver_id": 1004},
        {"driver_id": 1005},
    ],
).to_dict()

feature_vector

{'driver_id': [1004, 1005],
 'avg_daily_trips': [553, 762],
 'conv_rate': [0.7619121670722961, 0.45416760444641113],
 'acc_rate': [0.8696280121803284, 0.05302406847476959]}

## Просматр признаков с помощью веб-интерфейса
Просмотреть все зарегистрированные признаки, источники данных, сущности и службы можно с помощью веб-интерфейса.

In [ ]:
!uv run feast ui

![](./img/feast_ui.png)